#  Counting Email in a Database
## Counting Organizations

This application will read the mailbox data (mbox.txt) and count the number of email messages per organization (i.e. domain name of the email address) using a database with the following schema to maintain the counts.

```sql
CREATE TABLE Counts (org TEXT, count INTEGER)
```
When you have run the program on **mbox.txt** upload the resulting database file above for grading.

If you run the program multiple times in testing or with dfferent files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/emaildb.py.

The data file for this application is the same as in previous assignments: http://www.py4e.com/code3/mbox.txt.

Because the sample code is using an **UPDATE** statement and committing the results to the database as each record is read in the loop, it might take as long as a few minutes to process all the data. The commit insists on completely writing all the data to disk every time it is called.

The program can be speeded up greatly by moving the commit operation outside of the loop. In any database program, there is a balance between the number of operations you execute between commits and the importance of not losing the results of operations that have not yet been committed. 

In [19]:
import sqlite3

conn = sqlite3.connect('01-Counting_Email_in_a_Database.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox-short.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split('@')
    org = pieces[1].strip()
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()

Enter file name: mbox.txt
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


# Multi-Table Database - Tracks
## Musical Track Database

This application will read an iTunes export file in XML and produce a properly normalized database with this structure:
```sql
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
```
If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/tracks.zip. The ZIP file contains the **Library.xml** file to be used for this assignment. You can export your own tracks from iTunes and create a database, but for the database that you turn in for this assignment, only use the Library.xml data that is provided.

To grade this assignment, the program will run a query like this on your uploaded database and look for the data it expects to see:

```sql
SELECT Track.title, Artist.name, Album.title, Genre.name 
FROM Track JOIN Genre JOIN Album JOIN Artist 
ON Track.genre_id = Genre.ID and Track.album_id = Album.id AND Album.artist_id = Artist.id 
ORDER BY Artist.name LIMIT 3
```
The expected result of the modified query on your database is: (shown here as a simple HTML table with titles)

| title                                   | Artist | Album        | Genre |
|-----------------------------------------|--------|--------------|-------|
| Chase the Ace                           | AC/DC  | Who Made Who | Rock  |
| D.T.                                    | AC/DC  | Who Made Who | Rock  |
| For Those About To Rock (We Salute You) | AC/DC  | Who Made Who | Rock  |

In [9]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('02-Multi-Table_Database_Tracks.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
    
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER

);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')
    genre = lookup(entry, 'Genre')

    if name is None or artist is None or album is None or genre is None : 
        continue

    print(name, artist, album, count, rating, length, genre)
    
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', (genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, genre_id, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, genre_id, album_id, length, rating, count ) )

    conn.commit()


Enter file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103 Rock
Asche Zu Asche Rammstein Herzeleid 79 100 231810 Industrial
Beauty School Dropout Various Grease 48 100 239960 Soundtrack
Black Dog Led Zeppelin IV 109 100 296620 Rock
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118 Rock
Circles Bryan Lee Blues Is 54 60 355369 Funk
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130 Rock
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631 Rock
Electric Funeral Black Sabbath Paranoid 44 100 293015 Metal
Fat Bottomed Girls Queen Greatest Hits 38 100 257515 Rock
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750 Rock
Four Sticks Led Zeppelin IV 84 100 284421 Rock
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004 Soundtrack
Gelle Bryan Lee Blues Is 45 60 199836 Blues/R&B
Going To California Led Zeppelin IV 100 100 215666 Rock
Grease Various Grease 42 100 205792 Soundtrack
Hand of Doom Bl

The Immigrant Joanie Madden Natural Wonders Music Sampler 1999 350 None 362631 New Age
Pahrump-Big Water Cusco Natural Wonders Music Sampler 1999 342 None 191634 New Age
Seeker's Quest Cheryl Gunn Natural Wonders Music Sampler 1999 337 None 236434 New Age
Floating To Forever Dean Everson Natural Wonders Music Sampler 1999 337 None 351686 New Age
Open Road Jeff Bailey Relaxing Jazz 10 None 318563 Jazz
Ruby Kaiser Chiefs Yours Truly, Angry Mob 13 None 205008 Alternative & Punk
The Angry Mob Kaiser Chiefs Yours Truly, Angry Mob 41 None 288313 Alternative & Punk
Heat Dies Down Kaiser Chiefs Yours Truly, Angry Mob 12 None 237061 Alternative & Punk
Highroyds Kaiser Chiefs Yours Truly, Angry Mob 10 None 199575 Alternative & Punk
Love's Not A Competition (But I'm Winning) Kaiser Chiefs Yours Truly, Angry Mob 11 None 197799 Alternative & Punk
Thank You Very Much Kaiser Chiefs Yours Truly, Angry Mob 11 None 157753 Alternative & Punk
I Can Do It Without You Kaiser Chiefs Yours Truly, Angry Mob 11

Ian Horrocks: Standardizing OWL IEEE Computer Society Computing Conversations 1 None 572212 Podcast
Katie Hafner: The Origins of the Internet IEEE Computer Society Computing Conversations 1 None 652460 Podcast
Larry Smarr: Building Mosaic IEEE Computer Society Computing Conversations None None 894693 Podcast
Len Kleinrock: The Theory of Packets IEEE Computer Society Computing Conversations 1 None 674742 Podcast
Mitchell Baker: The Mozilla Foundation IEEE Computer Society Computing Conversations None None 994246 Podcast
Pooja Sankar: Building the Piazza Collaboration System IEEE Computer Society Computing Conversations None None 496404 Podcast
Van Jacobson: Content-Centric Networking IEEE Computer Society Computing Conversations None None 780251 Podcast
The Apache Software Foundation IEEE Computer Society Computing Conversations None None 542484 Podcast
A Brief History of Packets IEEE Computer Society Computing Conversations None None 1004643 Podcast
Discovering JavaScript Object Notati

Song for Someone U2 Songs of Innocence None None 226763 Rock
The Miracle (Of Joey Ramone) U2 Songs of Innocence None None 255382 Rock
Iris (Hold Me Close) U2 Songs of Innocence None None 319457 Rock
Goodbye Daughters of the Revolution The Black Crowes Warpaint None None 304147 Rock
Walk Believer Walk The Black Crowes Warpaint None None 280253 Rock
Choice Hotels International Undercover Boss Undercover Boss, Season 2 1 None 2614047 Reality TV
Microchip Jason Farnham YouTube Audio Library None None 114336 Dance & Electronic
Anant Agarwal IEEE Computer Society Computing Conversations None None 494000 Podcast


# Many Students in Many Courses

This application will read roster data in JSON format, parse the file, and then produce an SQLite database that contains a User, Course, and Member table and populate the tables from the data file.

You can base your solution on this code: http://www.py4e.com/code3/roster/roster.py - this code is incomplete as you need to modify the program to store the **role** column in the **Member** table to complete the assignment.

Each student gets their own file for the assignment. Download this file and save it as ```roster_data.json```. Move the downloaded file into the same folder as your ```roster.py``` program.

Once you have made the necessary changes to the program and it has been run successfully reading the above JSON data, run the following SQL command:
```sql
SELECT hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X
```
Find the first row in the resulting record set and enter the long string that looks like 53656C696E613333. 

In [11]:
import json
import sqlite3

conn = sqlite3.connect('03-Many_Students_in_Many_Courses.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = 'roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2]

    print((name, title, role))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )

    conn.commit()


Enter file name: 
('Abby', 'si110', 1)
('Kile', 'si110', 0)
('Maryk', 'si110', 0)
('Carra', 'si110', 0)
('Sinead', 'si110', 0)
('Kasra', 'si110', 0)
('Mirin', 'si110', 0)
('Rayann', 'si110', 0)
('Laird', 'si110', 0)
('Jun', 'si110', 0)
('Murdo', 'si110', 0)
('Dedeniseoluwa', 'si110', 0)
('Stefanie', 'si110', 0)
('Ana', 'si110', 0)
('Rhea', 'si110', 0)
('Arman', 'si110', 0)
('Cejay', 'si110', 0)
('Malcolm', 'si110', 0)
('Grant', 'si110', 0)
('Teagan', 'si110', 0)
('Lewie', 'si110', 0)
('Ceitidh', 'si106', 1)
('Andrei', 'si106', 0)
('Lucyanne', 'si106', 0)
('Mahum', 'si106', 0)
('Tiarn', 'si106', 0)
('Corrie', 'si106', 0)
('Monique', 'si106', 0)
('Maykayla', 'si106', 0)
('Rexford', 'si106', 0)
('Kaytie', 'si106', 0)
('Euan', 'si106', 0)
('Alyse', 'si106', 0)
('Strachan', 'si106', 0)
('Orley', 'si106', 0)
('Ceridwen', 'si106', 0)
('Ze', 'si106', 0)
('Moosa', 'si106', 0)
('Fenn', 'si106', 0)
('Aroob', 'si106', 0)
('Laurie', 'si106', 0)
('Hashem', 'si106', 0)
('Angeline', 'si106', 0)
('Lenn